In [99]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

import torch
print(torch.cuda.is_available())

import random

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

from skimage import io, transform
import numpy as np
import pandas as pd
import os
from PIL import Image

True


In [0]:
NUM_EPOCHS = 10
LR = 1e-5
BATCH_SIZE = 4
GAMMA = 0.95
curr_lr = LR

In [0]:
tfms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [0]:
categories = pd.read_csv("/content/training/solution.csv")["category"]
def getitem(idx):
    img_name = os.path.join("/content/training/training/", str(idx+1) + ".png")
    image = Image.open(img_name)
    if tfms:
        image = tfms(image)
    category = categories[idx]
    sample = {'image': image, 'category': category}

    return sample

In [0]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [0]:
model = models.resnet152(pretrained=True)

model = nn.Sequential(*list(model.children())[:-2],
                     nn.Conv2d(2048, 6, 3, 1),
                     nn.AdaptiveAvgPool2d(1), Flatten(),
                     nn.LogSoftmax())

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [113]:
print(model.children)

count = 0
for param in model.parameters():
    param.requires_grad = False
    count += 1
    if count >= 8:
        break

<bound method Module.children of Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stri

In [0]:
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [117]:
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    accuracy = torch.tensor(0)
    count = 0

    adjust_learning_rate(optimizer, curr_lr)
    curr_lr *= GAMMA
    
    optimizer.zero_grad()
    for c in range(1,1000):
        if c%BATCH_SIZE == 0:
            optimizer.step()
            optimizer.zero_grad()

        try:
            data = getitem(random.randint(0,4799))
        except:
            continue
        
        images = data["image"].unsqueeze(0)
        targets = torch.tensor([np.asscalar(data["category"])]).long()-1
        
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        
        outs = model(images)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()
                
        total_loss += loss.data
        accuracy += torch.sum(torch.argmax(outs, dim=1).cpu() == targets.cpu())
        count += 1

        if c%200 == 0:
            print("Epoch:", epoch, "Iter:", c, "Total loss:", total_loss, "Accuracy:", accuracy.cpu().numpy()/count)
    
    print("Training Epoch:", epoch, "Total loss:", total_loss)
    print("Training Epoch:", epoch, "Accuracy:", accuracy.cpu().numpy()/count)    
    
    total_loss = 0
    accuracy = 0

    for c in range(4799,4999):
      data = getitem(c)
      
      images = data["image"].unsqueeze(0)
      targets = torch.tensor([np.asscalar(data["category"])]).long()-1

      if torch.cuda.is_available():
          images = images.cuda()
          targets = targets.cuda()

      outs = model(images)
      loss = criterion(outs, targets)
      total_loss += loss.data
      accuracy += torch.sum(torch.argmax(outs, dim=1) == targets)
      
    print("Validation Epoch:", epoch, "Total loss:", total_loss)
    print("Validation Epoch:", epoch, "Accuracy:", accuracy.cpu().numpy()/200)
    #torch.save(model.state_dict(), "resnet152_model"+str(epoch)+".pt")
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 0 Iter: 200 Total loss: tensor(4.9839, device='cuda:0') Accuracy: 0.995
Epoch: 0 Iter: 400 Total loss: tensor(18.6289, device='cuda:0') Accuracy: 0.99
Epoch: 0 Iter: 600 Total loss: tensor(32.9282, device='cuda:0') Accuracy: 0.985
Epoch: 0 Iter: 800 Total loss: tensor(55.6489, device='cuda:0') Accuracy: 0.98125
Training Epoch: 0 Total loss: tensor(59.0905, device='cuda:0')
Training Epoch: 0 Accuracy: 0.983983983983984
Validation Epoch: 0 Total loss: tensor(17.0650, device='cuda:0')
Validation Epoch: 0 Accuracy: 0.975
Epoch: 1 Iter: 200 Total loss: tensor(4.4344, device='cuda:0') Accuracy: 0.995
Epoch: 1 Iter: 400 Total loss: tensor(17.7409, device='cuda:0') Accuracy: 0.99
Epoch: 1 Iter: 600 Total loss: tensor(28.3028, device='cuda:0') Accuracy: 0.99
Epoch: 1 Iter: 800 Total loss: tensor(42.5263, device='cuda:0') Accuracy: 0.9875
Training Epoch: 1 Total loss: tensor(45.5855, device='cuda:0')
Training Epoch: 1 Accuracy: 0.98998998998999
Validation Epoch: 1 Total loss: tensor(15.13

In [0]:
torch.save(model.state_dict(), "resnet152_model1.pt")

In [0]:
model.load_state_dict(torch.load("resnet152_model1.pt"))
model.eval()

In [0]:
def answer_builder(model):
  df = pd.DataFrame(columns=['id','category'])
  
  with torch.no_grad():
    for idx in range(1,40001):
      img_name = os.path.join("/content/testing/", str(idx) + ".png")
      image = Image.open(img_name)
      if tfms:
          image = tfms(image)
      image = image.unsqueeze(0).cuda()
      target = model(image)
      df.loc[idx] = [idx, torch.argmax(target).cpu().numpy()+1]
      if idx%1000 == 0:
        print(idx)

  df.to_csv('answer.csv', index=False)

In [0]:
answer_builder(model)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


1000
2000
3000
